# 🏀 UCSD Basketball XML → Smartabase CSV

This notebook processes raw NCAA box score XML files and converts **UCSD player data** into a **Smartabase-compatible CSV** file.

✅ Automatically maps to Smartabase field keys
✅ Only includes UCSD players
✅ Skips non-playing entries
✅ Runs directly in JupyterLite

In [ ]:
# 📂 Step 1: Upload XML file (e.g., 2640430.XML)
from IPython.display import display
print("⬆️ Use the file browser to upload your XML file to this folder (left sidebar in JupyterLite)")

In [ ]:
# ▶️ Step 2: Run the UCSD data extraction script
%run get_ucsd_data.py

In [ ]:
# 📥 Step 3: Download the generated CSV
from IPython.display import FileLink
FileLink("ucsd_smartabase_upload.csv")